In [21]:
using Distributions
using Turing
using Stan
using HDF5, JLD
using LinearAlgebra: dot

# Load data; loaded data is a list of dict named `ldastandata`
ldastandata = load(joinpath("data", "ldastandataV20K2M25L100.data"))["data"]
data = ldastandata[1]

K = data["K"]
V = data["V"]
M = data["M"]
N = data["N"]
w = data["w"]
doc = data["doc"]
beta = data["beta"]
alpha = data["alpha"]

vi = Turing.VarInfo()

# Load model
@model ldamodel(K, V, M, N, w, doc, beta, alpha) = begin
    theta = Vector{Vector{Real}}(undef, M)
    for m = 1:M
        theta[m] ~ Dirichlet(alpha)
    end

    phi = Vector{Vector{Real}}(undef, K)
    for k = 1:K
        phi[k] ~ Dirichlet(beta)
    end

    phi_dot_theta = [log.([dot(map(p -> p[i], phi), theta[m]) for i = 1:V]) for m=1:M]
    for n = 1:N
        Turing.acclogp!(vi, phi_dot_theta[doc[n]][w[n]])
    end
end

samples = sample(ldamodel(K, V, M, N, w, doc, beta, alpha), NUTS(1000, 0.65))

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /home/cameron/.julia/dev/Turing/src/inference/support/hmc_core.jl:235
[NUTS{Turing.Core.ForwardDiffAD{100},Any}] found initial ϵ: 1.6
└ @ Turing.Inference /home/cameron/.julia/dev/Turing/src/inference/support/hmc_core.jl:280
[NUTS] Sampling...  0%  ETA: 2:25:54
  ϵ:         1.6
  α:         0.4686505897073922
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...


[NUTS] Sampling...  0%  ETA: 1:23:39
  ϵ:         2.254840796848856
  α:         0.7913430629100163
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...


[NUTS] Sampling...  0%  ETA: 1:28:25
  ϵ:         2.544497689061883
  α:         2.2781099629742978e-11
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...


[NUTS] Sampling...  0%  ETA: 1:31:14
  ϵ:         0.347844116155567
  α:         0.9759399221853335
4m  pre_cond:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...


[NUTS] Sampling...  1%  ETA: 1:33:28
  ϵ:         0.7781988276304057
  α:         0.2135493148438453
4m  pre_co

DomainError: DomainError with log:
-2.220446049250313e-16 will only return a complex result if called with a complex argument. Try -2.220446049250313e-16(Complex(x)).

In [14]:
setchunksize(100)    # increase AD chunk-size to 100

MethodError: MethodError: no method matching Array{Array{Real,1},1}(::Int64)
Closest candidates are:
  Array{Array{Real,1},1}() where T at boot.jl:413
  Array{Array{Real,1},1}(!Matched::UndefInitializer, !Matched::Int64) where T at boot.jl:394
  Array{Array{Real,1},1}(!Matched::UndefInitializer, !Matched::Int64...) where {T, N} at boot.jl:400
  ...

In [ ]:
using Gadfly
using DataFrames

vis_topic_res(samples, K, V, avg_range) = begin
    phiarr = mean(samples[:phi][1:avg_range])

    phi = Matrix(0, V)
    for k = 1:K
        phi = vcat(phi, phiarr[k]')
    end

    df = DataFrame(Topic = vec(repmat(collect(1:K)', V, 1)),
                  Word = vec(repmat(collect(1:V)', 1, K)),
                  Probability = vec(phi))

    p = plot(df, x=:Word, y=:Topic, color=:Probability, Geom.rectbin)

    p
end